In [384]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [385]:
# Helper function to create dataframes for each trace output
def make_df(output_dir):
    stats = pd.DataFrame()
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        cache_info = []
        new_row = None
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if lines:
                for line in lines:
                    cache_info.append(int(line.split(':')[1]))
                        
                info = filename.split('_')
                new_row = pd.Series({
                    'set size': int(info[0]), 
                    'block size': int(info[1]),
                    'byte size': int(info[2]),
                    'write policy': info[3], 
                    'write through': info[4],
                    'cache type': info[5],
                    'total loads': cache_info[0],
                    'total stores': cache_info[1],
                    'load hits': cache_info[2],
                    'load misses': cache_info[3],
                    'store hits': cache_info[4],
                    'store misses': cache_info[5],
                    'total cycles': cache_info[6]
                })
                stats = pd.concat([stats, new_row.to_frame().T])
    stats['load hit/miss ratio'] = stats['load hits'] / stats['load misses']
    stats['store hit/miss ratio'] = stats['store hits'] / stats['store misses']
    stats['total hit/miss ratio'] =  (stats['load hits'] + stats['store hits']) / (stats['load misses'] + stats['store misses'])
    stats['cache size'] = stats['set size'] * stats['block size'] * stats['byte size']
    return stats    

In [386]:
gcc_dir = os.path.join(os.getcwd(), 'gcc_output')
gcc_data = make_df(gcc_dir)

swim_dir = os.path.join(os.getcwd(), 'swim_output')
swim_data = make_df(swim_dir)

In [387]:
print(f"Cache sizes:\n{np.sort(swim_data['cache size'].unique())}")
print(swim_data.keys())

Cache sizes:
[4 8 16 32 64 128 256 512 1024 2048 4096 8192 16384 32768 65536 131072
 262144 524288 1048576]
Index(['set size', 'block size', 'byte size', 'write policy', 'write through',
       'cache type', 'total loads', 'total stores', 'load hits', 'load misses',
       'store hits', 'store misses', 'total cycles', 'load hit/miss ratio',
       'store hit/miss ratio', 'total hit/miss ratio', 'cache size'],
      dtype='object')


In [388]:
def compute_hit_to_miss(cache_size, data):
    data.drop(columns=['total cycles', 'load hit/miss ratio',
       'store hit/miss ratio', 'cache size'])
    sorted_df_load = data.loc[data['cache size'] == cache_size].sort_values(by='load hit/miss ratio', ascending=False).iloc[0:3]
    sorted_df_store = data.loc[data['cache size'] == cache_size].sort_values(by='store hit/miss ratio', ascending=False).iloc[0:3]
    print(f"Top 3 rows for cache size {cache_size} with the highest load hit/miss ratio:")
    display(sorted_df_load)
    print(f"Top 3 rows for cache size {cache_size} with the highest store hit/miss ratio:")
    display(sorted_df_store)

In [389]:
test_cache_sizes = [1024, 4096, 16384, 65536, 262144]

In [390]:
# Compute stats for gcc 
for s in test_cache_sizes:
    compute_hit_to_miss(s, gcc_data)

Top 3 rows for cache size 1024 with the highest load hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,4,64,4,write-allocate,write-through,lru.txt,318197,197486,302689,15508,165198,32288,25076171,19.518249,5.11639,9.78925,1024
0,4,64,4,write-allocate,write-back,lru.txt,318197,197486,302689,15508,165198,32288,10096823,19.518249,5.11639,9.78925,1024
0,2,128,4,write-allocate,write-through,lru.txt,318197,197486,302683,15514,165145,32341,25082124,19.510313,5.106367,9.775948,1024


Top 3 rows for cache size 1024 with the highest store hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,2,16,32,write-allocate,write-back,lru.txt,318197,197486,295667,22530,188419,9067,51076848,13.123258,20.780743,15.320632,1024
0,2,16,32,write-allocate,write-through,lru.txt,318197,197486,295667,22530,188419,9067,45550950,13.123258,20.780743,15.320632,1024
0,1,32,32,write-allocate,write-through,lru.txt,318197,197486,295815,22382,188350,9136,45487819,13.216647,20.616243,15.361539,1024


Top 3 rows for cache size 4096 with the highest load hit/miss ratio:


...

,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,16,128,32,write-allocate,write-through,lru.txt,318197,197486,316466,1731,192914,4572,25311255,182.822646,42.194663,80.815485,65536
0,16,128,32,write-allocate,write-back,lru.txt,318197,197486,316466,1731,192914,4572,8966338,182.822646,42.194663,80.815485,65536
0,32,64,32,write-allocate,write-back,lru.txt,318197,197486,316467,1730,192913,4573,8966338,182.928902,42.185218,80.815485,65536


Top 3 rows for cache size 262144 with the highest load hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,64,128,32,write-allocate,write-back,lru.txt,318197,197486,316694,1503,192955,4531,5342883,210.707917,42.585522,84.462877,262144
0,64,128,32,write-allocate,write-through,fifo.txt,318197,197486,316694,1503,192955,4531,25096014,210.707917,42.585522,84.462877,262144
0,64,128,32,write-allocate,write-through,lru.txt,318197,197486,316694,1503,192955,4531,25096014,210.707917,42.585522,84.462877,262144


Top 3 rows for cache size 262144 with the highest store hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,128,64,32,write-allocate,write-back,lru.txt,318197,197486,316692,1505,192955,4531,5361304,210.426578,42.585522,84.434559,262144
0,64,128,32,write-allocate,write-back,lru.txt,318197,197486,316694,1503,192955,4531,5342883,210.707917,42.585522,84.462877,262144
0,64,128,32,write-allocate,write-back,fifo.txt,318197,197486,316694,1503,192955,4531,5342883,210.707917,42.585522,84.462877,262144


In [391]:
for s in test_cache_sizes:
    compute_hit_to_miss(s, swim_data)

Top 3 rows for cache size 1024 with the highest load hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,4,8,32,no-write-allocate,write-through,lru.txt,220668,82525,210989,9679,59939,22586,16298893,21.798636,2.653812,8.396963,1024
0,1,32,32,no-write-allocate,write-through,lru.txt,220668,82525,210926,9742,59790,22735,16349293,21.651201,2.629866,8.335622,1024
0,2,16,32,no-write-allocate,write-through,lru.txt,220668,82525,210776,9892,59934,22591,16469293,21.307723,2.653003,8.333898,1024


Top 3 rows for cache size 1024 with the highest store hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,2,16,32,write-allocate,write-through,lru.txt,220668,82525,209284,11384,73724,8801,24712494,18.384048,8.376775,14.020708,1024
0,2,16,32,write-allocate,write-back,lru.txt,220668,82525,209284,11384,73724,8801,32593746,18.384048,8.376775,14.020708,1024
0,1,32,32,write-allocate,write-through,lru.txt,220668,82525,208886,11782,73701,8824,25049317,17.729248,8.352335,13.713821,1024


Top 3 rows for cache size 4096 with the highest load hit/miss ratio:


...

,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,32,64,32,write-allocate,write-back,lru.txt,220668,82525,220014,654,76708,5817,9022816,336.412844,13.186866,45.854118,65536
0,128,16,32,write-allocate,write-through,lru.txt,220668,82525,220014,654,76708,5817,13738310,336.412844,13.186866,45.854118,65536
0,16,128,32,write-allocate,write-through,lru.txt,220668,82525,220014,654,76708,5817,13738310,336.412844,13.186866,45.854118,65536


Top 3 rows for cache size 262144 with the highest load hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,128,64,32,write-allocate,write-back,lru.txt,220668,82525,220077,591,76720,5805,5419993,372.380711,13.216193,46.403533,262144
0,64,128,32,write-allocate,write-through,lru.txt,220668,82525,220077,591,76720,5805,13678298,372.380711,13.216193,46.403533,262144
0,256,32,32,write-allocate,write-back,fifo.txt,220668,82525,220077,591,76720,5805,5421595,372.380711,13.216193,46.403533,262144


Top 3 rows for cache size 262144 with the highest store hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,128,64,32,write-allocate,write-back,lru.txt,220668,82525,220077,591,76720,5805,5419993,372.380711,13.216193,46.403533,262144
0,64,128,32,write-allocate,write-through,lru.txt,220668,82525,220077,591,76720,5805,13678298,372.380711,13.216193,46.403533,262144
0,256,32,32,write-allocate,write-back,fifo.txt,220668,82525,220077,591,76720,5805,5421595,372.380711,13.216193,46.403533,262144


In [392]:
def compute_best_overall(cache_size, data):
    data.drop(columns=['total cycles', 'load hit/miss ratio',
       'store hit/miss ratio', 'cache size'])
    sorted_df = data.loc[data['cache size'] == cache_size].sort_values(by='total hit/miss ratio', ascending=False).iloc[0:3]
    print(f"Top 3 rows for cache size {cache_size} with the highest total hit/miss ratio:")
    display(sorted_df)

In [393]:
for s in test_cache_sizes:
    compute_best_overall(s, gcc_data)

Top 3 rows for cache size 1024 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,1,32,32,write-allocate,write-through,lru.txt,318197,197486,295815,22382,188350,9136,45487819,13.216647,20.616243,15.361539,1024
0,1,32,32,write-allocate,write-back,lru.txt,318197,197486,295815,22382,188350,9136,50950369,13.216647,20.616243,15.361539,1024
0,1,64,16,write-allocate,write-back,lru.txt,318197,197486,300027,18170,184063,13423,25796012,16.512218,13.712508,15.322698,1024


Top 3 rows for cache size 4096 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,1,128,32,write-allocate,write-through,lru.txt,318197,197486,314052,4145,192202,5284,27812767,75.766466,36.374338,53.691166,4096
0,1,128,32,write-allocate,write-back,lru.txt,318197,197486,314052,4145,192202,5284,15508984,75.766466,36.374338,53.691166,4096
0,2,64,32,write-allocate,write-back,lru.txt,318197,197486,314038,4159,192204,5282,15528196,75.508055,36.388489,53.62165,4096


Top 3 rows for cache size 16384 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,4,128,32,write-allocate,write-back,lru.txt,318197,197486,315884,2313,192689,4797,11488681,136.568958,40.168647,71.529255,16384
0,4,128,32,write-allocate,write-through,lru.txt,318197,197486,315884,2313,192689,4797,25957080,136.568958,40.168647,71.529255,16384
0,8,64,32,write-allocate,write-back,lru.txt,318197,197486,315869,2328,192690,4796,11511095,135.68256,40.177231,71.386721,16384


Top 3 rows for cache size 65536 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,16,128,32,write-allocate,write-through,lru.txt,318197,197486,316466,1731,192914,4572,25311255,182.822646,42.194663,80.815485,65536
0,32,64,32,write-allocate,write-back,lru.txt,318197,197486,316467,1730,192913,4573,8966338,182.928902,42.185218,80.815485,65536
0,32,64,32,write-allocate,write-through,lru.txt,318197,197486,316467,1730,192913,4573,25311256,182.928902,42.185218,80.815485,65536


Top 3 rows for cache size 262144 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,64,128,32,write-allocate,write-back,lru.txt,318197,197486,316694,1503,192955,4531,5342883,210.707917,42.585522,84.462877,262144
0,64,128,32,write-allocate,write-through,fifo.txt,318197,197486,316694,1503,192955,4531,25096014,210.707917,42.585522,84.462877,262144
0,64,128,32,write-allocate,write-through,lru.txt,318197,197486,316694,1503,192955,4531,25096014,210.707917,42.585522,84.462877,262144


In [394]:
for s in test_cache_sizes:
    compute_best_overall(s, swim_data)

Top 3 rows for cache size 1024 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,4,8,32,write-allocate,write-back,lru.txt,220668,82525,209409,11259,73683,8842,32459262,18.599254,8.333296,14.083478,1024
0,4,8,32,write-allocate,write-through,lru.txt,220668,82525,209409,11259,73683,8842,24645335,18.599254,8.333296,14.083478,1024
0,2,16,32,write-allocate,write-back,lru.txt,220668,82525,209284,11384,73724,8801,32593746,18.384048,8.376775,14.020708,1024


Top 3 rows for cache size 4096 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,8,16,32,write-allocate,write-through,lru.txt,220668,82525,216854,3814,76149,6376,16714069,56.857368,11.943068,28.753974,4096
0,8,16,32,write-allocate,write-back,lru.txt,220668,82525,216854,3814,76149,6376,16514855,56.857368,11.943068,28.753974,4096
0,4,32,32,write-allocate,write-through,lru.txt,220668,82525,216814,3854,76167,6358,16731651,56.256876,11.979711,28.689875,4096


Top 3 rows for cache size 16384 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,8,64,32,write-allocate,write-through,lru.txt,220668,82525,219922,746,76682,5843,13832736,294.801609,13.123738,45.015025,16384
0,8,64,32,write-allocate,write-back,lru.txt,220668,82525,219922,746,76682,5843,10442070,294.801609,13.123738,45.015025,16384
0,4,128,32,write-allocate,write-through,lru.txt,220668,82525,219922,746,76680,5845,13834338,294.801609,13.118905,45.001062,16384


Top 3 rows for cache size 65536 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,32,64,32,write-allocate,write-back,lru.txt,220668,82525,220014,654,76708,5817,9022816,336.412844,13.186866,45.854118,65536
0,128,16,32,write-allocate,write-back,lru.txt,220668,82525,220014,654,76708,5817,9022816,336.412844,13.186866,45.854118,65536
0,128,16,32,write-allocate,write-through,lru.txt,220668,82525,220014,654,76708,5817,13738310,336.412844,13.186866,45.854118,65536


Top 3 rows for cache size 262144 with the highest total hit/miss ratio:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,128,64,32,write-allocate,write-back,lru.txt,220668,82525,220077,591,76720,5805,5419993,372.380711,13.216193,46.403533,262144
0,64,128,32,write-allocate,write-through,lru.txt,220668,82525,220077,591,76720,5805,13678298,372.380711,13.216193,46.403533,262144
0,256,32,32,write-allocate,write-back,fifo.txt,220668,82525,220077,591,76720,5805,5421595,372.380711,13.216193,46.403533,262144


In [395]:
def compute_best_speed(cache_size, data):
    sorted_df = data.loc[data['cache size'] == cache_size].sort_values(by='total cycles', ascending=True).iloc[0:3]
    sorted_df.drop(columns=['load hit/miss ratio',
       'store hit/miss ratio', 'cache size'])
    print(f"Top 3 rows for cache size {cache_size} with the lowest number of total cycles:")
    display(sorted_df)

In [396]:
for s in test_cache_sizes:
    compute_best_speed(s, gcc_data)

Top 3 rows for cache size 1024 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,4,64,4,write-allocate,write-back,lru.txt,318197,197486,302689,15508,165198,32288,10096823,19.518249,5.11639,9.78925,1024
0,2,128,4,write-allocate,write-back,lru.txt,318197,197486,302683,15514,165145,32341,10108682,19.510313,5.106367,9.775948,1024
0,8,32,4,write-allocate,write-back,lru.txt,318197,197486,302403,15794,165124,32362,10169183,19.146701,5.102404,9.708593,1024


Top 3 rows for cache size 4096 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,8,128,4,write-allocate,write-back,lru.txt,318197,197486,310411,7786,168739,28747,7755392,39.86784,5.869795,13.115539,4096
0,32,32,4,write-allocate,write-back,lru.txt,318197,197486,310403,7794,168743,28743,7756196,39.825892,5.870751,13.113994,4096
0,128,8,4,write-allocate,write-back,lru.txt,318197,197486,310361,7836,168757,28729,7761824,39.60707,5.874099,13.103186,4096


Top 3 rows for cache size 16384 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,32,128,4,write-allocate,write-back,lru.txt,318197,197486,312768,5429,169754,27732,6767348,57.61061,6.121232,14.550888,16384
0,64,64,4,write-allocate,write-back,lru.txt,318197,197486,312765,5432,169749,27737,6768956,57.57824,6.119948,14.547137,16384
0,128,32,4,write-allocate,write-back,lru.txt,318197,197486,312763,5434,169702,27784,6778805,57.55668,6.107904,14.524204,16384


Top 3 rows for cache size 65536 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,128,128,4,write-allocate,write-back,lru.txt,318197,197486,313344,4853,170076,27410,5345762,64.567072,6.204889,14.983727,65536
0,256,64,4,write-allocate,write-back,lru.txt,318197,197486,313345,4852,170074,27412,5345963,64.580585,6.204363,14.983232,65536
0,256,64,4,write-allocate,write-back,fifo.txt,318197,197486,313173,5024,169984,27502,5398625,62.33539,6.180787,14.854486,65536


Top 3 rows for cache size 262144 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,256,128,8,write-allocate,write-back,lru.txt,318197,197486,314571,3626,180111,17375,4715883,86.754275,10.366101,23.555164,262144
0,256,128,8,write-allocate,write-back,fifo.txt,318197,197486,314571,3626,180111,17375,4715883,86.754275,10.366101,23.555164,262144
0,128,128,16,write-allocate,write-back,lru.txt,318197,197486,315855,2342,188617,8869,5000083,134.8655,21.266997,44.997948,262144


In [397]:
for s in test_cache_sizes:
    compute_best_speed(s, swim_data)

Top 3 rows for cache size 1024 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,2,128,4,write-allocate,write-back,lru.txt,220668,82525,204923,15745,58638,23887,8243369,13.015116,2.454808,6.650207,1024
0,4,64,4,write-allocate,write-back,lru.txt,220668,82525,204882,15786,58360,24165,8307488,12.978715,2.415063,6.589122,1024
0,8,32,4,write-allocate,write-back,lru.txt,220668,82525,204777,15891,58088,24437,8383265,12.886351,2.377051,6.518176,1024


Top 3 rows for cache size 4096 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,8,128,4,write-allocate,write-back,lru.txt,220668,82525,214399,6269,61227,21298,5746796,34.199872,2.874777,9.998404,4096
0,32,32,4,write-allocate,write-back,lru.txt,220668,82525,214410,6258,61216,21309,5749624,34.261745,2.872777,9.998404,4096
0,16,64,4,write-allocate,write-back,lru.txt,220668,82525,214399,6269,61226,21299,5749825,34.199872,2.874595,9.998005,4096


Top 3 rows for cache size 16384 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,32,128,4,write-allocate,write-back,lru.txt,220668,82525,218929,1739,61624,20901,4565377,125.893617,2.948376,12.391917,16384
0,64,64,4,write-allocate,write-back,lru.txt,220668,82525,218929,1739,61622,20903,4565779,125.893617,2.947998,12.390734,16384
0,128,32,4,write-allocate,write-back,lru.txt,220668,82525,218928,1740,61618,20907,4566784,125.82069,2.947243,12.387778,16384


Top 3 rows for cache size 65536 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,256,64,4,write-allocate,write-back,lru.txt,220668,82525,219046,1622,61666,20859,3757738,135.046856,2.956326,12.486633,65536
0,128,128,4,write-allocate,write-back,lru.txt,220668,82525,219045,1623,61664,20861,3758341,134.963031,2.955947,12.484834,65536
0,128,128,4,write-allocate,write-back,fifo.txt,220668,82525,218961,1707,61627,20898,3782662,128.272408,2.948942,12.412652,65536


Top 3 rows for cache size 262144 with the lowest number of total cycles:


,set size,block size,byte size,write policy,write through,cache type,total loads,total stores,load hits,load misses,store hits,store misses,total cycles,load hit/miss ratio,store hit/miss ratio,total hit/miss ratio,cache size
0,256,128,8,write-allocate,write-back,lru.txt,220668,82525,219354,1314,64023,18502,4266393,166.936073,3.460329,14.300414,262144
0,256,128,8,write-allocate,write-back,fifo.txt,220668,82525,219354,1314,64023,18502,4266393,166.936073,3.460329,14.300414,262144
0,256,64,16,write-allocate,write-back,lru.txt,220668,82525,219742,926,72028,10497,4872393,237.302376,6.86177,25.542327,262144
